In [19]:
import csv
import requests
import re

In [1]:
params = {
    "part": "snippet",
    "videoId": input("Id do vídeo: "),
    "key": input("Chave de api: "),
    "maxResults": 100
}

In [21]:
comments = []
next_page_token = None

In [22]:
while True:
  if next_page_token:
    params["pageToken"] = next_page_token

  response = requests.get("https://www.googleapis.com/youtube/v3/commentThreads", params=params)
  data = response.json()

  if "items" not in data:
    print("Nenhum comentário encontrado")
    break

  for item in data["items"]:
    comment = item["snippet"]["topLevelComment"]
    author = comment["snippet"]["authorDisplayName"]
    message = comment["snippet"]["textDisplay"]
    like_count = comment["snippet"]["likeCount"]
    time = comment["snippet"]["publishedAt"]

    comments.append({"author": author, "message": message, "like_count": like_count, "time": time})

    next_page_token = data.get("nextPageToken")
  if not next_page_token:
    break

In [23]:
top_100_comments = sorted(comments, key=lambda x: x["like_count"], reverse=True)[:100]

for comment in top_100_comments:
  comment["message"] = comment["message"].replace("\n", " ")
  comment["message"] = comment["message"].replace("\r", " ")
  comment["message"] = comment["message"].replace('"', "'")
  comment["message"] = comment["message"].replace("&quot;", "'")
  comment["message"] = comment["message"].replace(";", ".")
  comment["message"] = re.sub(r'<[^>]+>', '', comment["message"])

In [24]:
output_file = "youtube_comments.csv"

with open(output_file, "w", encoding="utf-8-sig", newline="") as csv_file:
  writer = csv.DictWriter(csv_file, fieldnames=['author', 'message', 'like_count', 'time'], delimiter=";")
  writer.writeheader()
  writer.writerows(top_100_comments)